# notebook for 0905 final dataset

In [1]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
tqdm.pandas()
import torch
from torch import nn
from torch.nn import Transformer
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.metrics import average_precision_score


In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

# Now, you can use `device` to send your tensors to the GPU or CPU.


2.0.1
11.7
Using GPU: NVIDIA A100-SXM4-40GB


In [3]:
import torch

# Print available GPUs
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


GPU 0: NVIDIA A100-SXM4-40GB
GPU 1: NVIDIA A100-SXM4-40GB


In [4]:
device

device(type='cuda')

In [5]:
import os
import psutil
def memory_usage():
    process = psutil.Process(os.getpid())
    total_memory = psutil.virtual_memory().total / (1024**3)  # total RAM in GB
    used_memory = total_memory - process.memory_info().rss / (1024**3)  # used RAM in GB
    available_memory = psutil.virtual_memory().available / (1024**3)  # available RAM in GB
    return f"Total Memory: {total_memory:.2f} GB | Used Memory: {used_memory:.2f} GB | Available Memory: {available_memory:.2f} GB"

print(memory_usage())

Total Memory: 1007.45 GB | Used Memory: 1007.07 GB | Available Memory: 988.72 GB


In [6]:

# Custom Dataset
class MyDataset(Dataset):
    def __init__(self, src_data, tgt_data):
        self.src_data = [torch.tensor(seq, dtype=torch.float32) for seq in src_data]
        self.tgt_data = [torch.tensor(tgt, dtype=torch.float32) for tgt in tgt_data]

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        return self.src_data[idx], self.tgt_data[idx]

# Custom Collate function to handle variable sequence length
def collate_fn(batch):
    src, tgt = zip(*batch)
    src = pad_sequence(src, batch_first=True, padding_value=0)
    tgt = pad_sequence(tgt, batch_first=True, padding_value=-1)  # Special padding value for target data
    return src, tgt


def create_masks(src):
    src_key_padding_mask = (src == 0).all(axis=-1)
    
    # future_mask shape should be (seq_len, seq_len)
    future_mask = torch.triu(torch.ones((src.shape[1], src.shape[1])), diagonal=1).bool()

    return src_key_padding_mask, future_mask



In [7]:
# Transformer KG Model
# first KG_dims are word embedding vectors, will be put into a linear layer first to reduce the dim to KG_compress_dims
# the compressed word embedding will be concated with the other features , and go through transformers 
class TransformerKGModel(nn.Module):
    def __init__(self, KG_dims, KG_compress_dims, input_dim, embedding_size, num_heads, num_layers, dropout):
        super(TransformerKGModel, self).__init__()
        self.compress  = nn.Linear(KG_dims, KG_compress_dims)
        #self.concat_dims = KG_compress_dims
        self.embedding = nn.Linear(input_dim, embedding_size)
        self.transformer = Transformer(
            d_model=embedding_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dropout=dropout,
        )
        self.fc = nn.Linear(embedding_size, 1)

    def forward(self, src, src_key_padding_mask=None, future_mask=None):
        
        KG_part = src[:,:,:self.compress.in_features] # first KG_dims features are word embeddings
        
        
        other_parts = src[:,:, self.compress.in_features:] # [seq_len, batch_size, feature_size]
        # Compress KG_part  to KG_compress_dims
        compressed = self.compress(KG_part)  
        # Concatenate with second part
        src = torch.cat((compressed, other_parts), dim=-1)  
        # Reshape for transformer
   
        #src = src.unsqueeze(1)
        

        src = self.embedding(src)

        src = src.transpose(0, 1) 

        output = self.transformer.encoder(src, src_key_padding_mask=src_key_padding_mask, mask = future_mask)

        output = self.fc(output)

        output = torch.sigmoid(output)

        return output.transpose(0, 1).squeeze(-1)


In [8]:
import sklearn.metrics as skmetrics
def plot_roc(y_test, y_pred_prob, pos_label=True):
    """
    Function to plot the ROC curve and display the AUC of 
    this predictor.
    """
    fpr, tpr, _ = skmetrics.roc_curve(y_test, y_pred_prob, pos_label=pos_label)
    roc_auc = skmetrics.auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(
        fpr[2],
        tpr[2],
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc,
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.plot(fpr, tpr)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic example")
    plt.legend(loc="lower right")
    plt.show()
    

## data processing

In [34]:
# read study data
dataset = pd.read_csv("./data/new_data_sample_0905.csv", sep=',', index_col=None)


In [35]:
len(set(dataset.uid))

39931

In [36]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36452215 entries, 0 to 36452214
Data columns (total 18 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   uid                               object 
 1   register_time                     object 
 2   wid                               object 
 3   spelling                          object 
 4   difficulty                        int64  
 5   review_time                       object 
 6   response                          int64  
 7   study_method                      int64  
 8   repeat_time_inday                 int64  
 9   real_interval_history_byday       object 
 10  last_real_interval_byday          int64  
 11  real_delta_interval_within_inday  int64  
 12  review_day_th                     int64  
 13  delta_t_between_inday             float64
 14  delta_t_within_inday              float64
 15  repeat_time_history               int64  
 16  delta_interval_within_inday       

In [37]:
 # del dataset['repeat_time_inday']information leakage 

In [38]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36452215 entries, 0 to 36452214
Data columns (total 18 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   uid                               object 
 1   register_time                     object 
 2   wid                               object 
 3   spelling                          object 
 4   difficulty                        int64  
 5   review_time                       object 
 6   response                          int64  
 7   study_method                      int64  
 8   repeat_time_inday                 int64  
 9   real_interval_history_byday       object 
 10  last_real_interval_byday          int64  
 11  real_delta_interval_within_inday  int64  
 12  review_day_th                     int64  
 13  delta_t_between_inday             float64
 14  delta_t_within_inday              float64
 15  repeat_time_history               int64  
 16  delta_interval_within_inday       

### only use the words in the book list, there are a lot of phrases in the study logs , but they only take a very small proprotion in the dataset

In [39]:
len(set(dataset.wid))

24640

In [40]:
# Ensure the column is of string type
dataset['spelling'] = dataset['spelling'].astype(str)

# Replace NaNs with a placeholder string
dataset['spelling'] = dataset['spelling'].fillna('')

#sub_dataset = dataset[~dataset.spelling.str.contains(' ')]

In [41]:
word_net_diffculty_dataset = pd.read_csv('./data/tmp_word_net_diffculty_dataset.tsv',sep='\t')
#word_net_diffculty_dataset = word_net_diffculty_dataset.drop_duplicates()
word_net_diffculty_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13428 entries, 0 to 13427
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   wid        13428 non-null  object 
 1   spelling   13428 non-null  object 
 2   rsp1_rate  13428 non-null  float64
 3   rsp2_rate  13428 non-null  float64
 4   rsp3_rate  13428 non-null  float64
 5   rsp4_rate  13428 non-null  float64
dtypes: float64(4), object(2)
memory usage: 629.6+ KB


In [42]:
word_net_acknowledge_dataset = pd.read_csv('./data/tmp_word_net_acknowledge_dataset.tsv',sep='\t')
word_net_acknowledge_dataset = word_net_acknowledge_dataset.drop_duplicates()
word_net_acknowledge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6709 entries, 0 to 11647
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   wid               6709 non-null   object 
 1   acknowledge_rate  6709 non-null   float64
dtypes: float64(1), object(1)
memory usage: 157.2+ KB


In [43]:
word_net_acknowledge_dataset.head()

,wid,acknowledge_rate
0,57067bb0a172044907c61dc0,0.318209
1,57067ba5a172044907c6195b,0.560077
2,57067ba7a172044907c61a8a,0.342722
3,57067b8da172044907c60feb,0.707940
4,57067ba7a172044907c61afa,0.285558


In [44]:
word_net_diffculty_ackknowledge_dataset = pd.merge(word_net_diffculty_dataset,word_net_acknowledge_dataset,on=['wid'],how='inner')
word_net_diffculty_ackknowledge_dataset = word_net_diffculty_ackknowledge_dataset.drop_duplicates()
word_net_diffculty_ackknowledge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6709 entries, 0 to 13427
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   wid               6709 non-null   object 
 1   spelling          6709 non-null   object 
 2   rsp1_rate         6709 non-null   float64
 3   rsp2_rate         6709 non-null   float64
 4   rsp3_rate         6709 non-null   float64
 5   rsp4_rate         6709 non-null   float64
 6   acknowledge_rate  6709 non-null   float64
dtypes: float64(5), object(2)
memory usage: 419.3+ KB


In [45]:
word_net_diffculty_ackknowledge_dataset.head()

,wid,spelling,rsp1_rate,rsp2_rate,rsp3_rate,rsp4_rate,acknowledge_rate
0,57067bb0a172044907c61dc0,preach,0.3624,0.1708,0.4665,0.0003,0.318209
2,57067ba5a172044907c6195b,arise,0.5498,0.1521,0.2977,0.0004,0.560077
4,57067ba7a172044907c61a8a,ingredient,0.3437,0.1730,0.4831,0.0003,0.342722
6,57067b8da172044907c60feb,application,0.6840,0.1079,0.2077,0.0005,0.707940
8,57067ba7a172044907c61afa,ingenious,0.4271,0.1814,0.3912,0.0003,0.285558


In [46]:
dataset = pd.merge(dataset,word_net_diffculty_ackknowledge_dataset, on=['wid','spelling'],how='inner')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36301382 entries, 0 to 36301381
Data columns (total 23 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   uid                               object 
 1   register_time                     object 
 2   wid                               object 
 3   spelling                          object 
 4   difficulty                        int64  
 5   review_time                       object 
 6   response                          int64  
 7   study_method                      int64  
 8   repeat_time_inday                 int64  
 9   real_interval_history_byday       object 
 10  last_real_interval_byday          int64  
 11  real_delta_interval_within_inday  int64  
 12  review_day_th                     int64  
 13  delta_t_between_inday             float64
 14  delta_t_within_inday              float64
 15  repeat_time_history               int64  
 16  delta_interval_within_inday       

### add KG embedding and filter thw word sets again

In [47]:
# add KG embedding and filter thw word sets again
KGembedding = pd.read_csv('./data/KGembeddings.csv')
KGembedding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6685 entries, 0 to 6684
Data columns (total 65 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   w        6685 non-null   object 
 1   embed1   6685 non-null   float64
 2   embed2   6685 non-null   float64
 3   embed3   6685 non-null   float64
 4   embed4   6685 non-null   float64
 5   embed5   6685 non-null   float64
 6   embed6   6685 non-null   float64
 7   embed7   6685 non-null   float64
 8   embed8   6685 non-null   float64
 9   embed9   6685 non-null   float64
 10  embed10  6685 non-null   float64
 11  embed11  6685 non-null   float64
 12  embed12  6685 non-null   float64
 13  embed13  6685 non-null   float64
 14  embed14  6685 non-null   float64
 15  embed15  6685 non-null   float64
 16  embed16  6685 non-null   float64
 17  embed17  6685 non-null   float64
 18  embed18  6685 non-null   float64
 19  embed19  6685 non-null   float64
 20  embed20  6685 non-null   float64
 21  embed21  6685 

In [48]:
KGembedding.rename(columns={'w':'spelling'},inplace=True)
KGembedding.head()

,spelling,embed1,embed2,embed3,embed4,embed5,embed6,embed7,embed8,embed9,...,embed55,embed56,embed57,embed58,embed59,embed60,embed61,embed62,embed63,embed64
0,a,-0.112005,0.175023,0.081303,0.119429,0.234358,-0.285393,0.088960,-0.084222,-0.146858,...,0.121453,0.046255,0.027163,0.253145,0.250952,0.395841,0.263606,-0.024474,-0.125878,-0.034926
1,abandon,-0.400461,0.727612,0.237638,0.407411,1.023871,-1.093308,0.371002,-0.282474,-0.603780,...,0.572360,0.213719,0.139102,0.971377,0.966684,1.702201,1.043585,-0.034144,-0.499250,-0.070479
2,abdomen,-0.485593,1.103594,0.264719,0.511833,1.521983,-1.432766,0.503473,-0.349383,-0.807522,...,0.910728,0.306264,0.270944,1.298496,1.351456,2.541386,1.477359,0.069351,-0.689769,-0.026549
3,abide,-0.479631,1.098984,0.261993,0.506807,1.512208,-1.418678,0.499232,-0.345371,-0.799597,...,0.907186,0.303954,0.270819,1.286539,1.342442,2.527811,1.467622,0.073551,-0.683922,-0.024235
4,ability,-0.257335,0.465007,0.156881,0.264888,0.649716,-0.698882,0.233898,-0.185201,-0.381730,...,0.361201,0.133533,0.090380,0.622238,0.620555,1.080868,0.667867,-0.024280,-0.319865,-0.049535


In [49]:
dataset = pd.merge(KGembedding,dataset, on=['spelling'],how='inner')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34690422 entries, 0 to 34690421
Data columns (total 87 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   spelling                          object 
 1   embed1                            float64
 2   embed2                            float64
 3   embed3                            float64
 4   embed4                            float64
 5   embed5                            float64
 6   embed6                            float64
 7   embed7                            float64
 8   embed8                            float64
 9   embed9                            float64
 10  embed10                           float64
 11  embed11                           float64
 12  embed12                           float64
 13  embed13                           float64
 14  embed14                           float64
 15  embed15                           float64
 16  embed16                           

### check the distribution of word review counts per user


In [50]:
data_gp = dataset.groupby(['uid']).count().sort_values(by='register_time')


percentiles_list = list(range(0, 101, 10)) + [1,5,95 ,99]  # includes 1st and 99th percentile
percentiles_list.sort()  # make sure the percentiles are in ascending order

percentiles = {i: data_gp['wid'].quantile(i/100) for i in percentiles_list}

# Now percentiles is a dictionary that contains the 0th, 1st, 10th, 20th, ..., 99th, 100th percentile of 'column_name'

for percentile in percentiles_list:
    print(f'{percentile} percentile: {percentiles[percentile]}')
    

0 percentile: 1.0
1 percentile: 3.0
5 percentile: 20.0
10 percentile: 30.0
20 percentile: 73.0
30 percentile: 148.0
40 percentile: 266.0
50 percentile: 437.0
60 percentile: 639.0
70 percentile: 921.0
80 percentile: 1364.0
90 percentile: 2265.0
95 percentile: 3266.8499999999985
99 percentile: 5761.549999999952
100 percentile: 27652.0


In [51]:
# only valid users : filtering out abnormal users
valid_uids = list(data_gp[(data_gp.register_time<=3000)&(data_gp.register_time>=30)].index)
len(valid_uids)

33700

In [52]:
#subselect the dataset which only has valid users
dataset= dataset[dataset.uid.isin(valid_uids)]
dataset["review_time"] = pd.to_datetime(dataset["review_time"] )
dataset["register_time"] = pd.to_datetime(dataset["register_time"] )
dataset = dataset.sort_values(by=['uid', 'review_time'])

In [53]:
# process Y
dataset['Y'] = dataset.response.progress_apply(lambda x : 1 if x ==1 else 0)

# lower memory
for column in dataset.select_dtypes(include=['int64']):
    dataset[column] = dataset[column].astype('int8')
for column in dataset.select_dtypes(include=['float64']):
    dataset[column] = dataset[column].astype('float32')

100%|██████████| 23656968/23656968 [00:12<00:00, 1924209.04it/s]


In [54]:
valid_word = list(set(dataset['spelling'].values))
len(valid_word)

6180

### create the last response(0/1) as an input feature for next time step. For the first-time review's last response , use the acknowledge rate for the word instead

In [55]:
def get_last_response(x):
    response_history = x[0]
    acknowledge_rate = x[1]
    try:
        last_response = int(response_history[-2])
        if last_response ==1 :
            None 
        else:
            last_response=0
    except : 
        last_response = int(acknowledge_rate>0.5)
    return last_response

In [56]:
dataset['last_response'] = dataset[['response_history','acknowledge_rate']].progress_apply(get_last_response, axis =1)

100%|██████████| 23656968/23656968 [01:28<00:00, 266848.96it/s]


In [57]:
dataset.head()

,spelling,embed1,embed2,embed3,embed4,embed5,embed6,embed7,embed8,embed9,...,repeat_time_history,delta_interval_within_inday,response_history,rsp1_rate,rsp2_rate,rsp3_rate,rsp4_rate,acknowledge_rate,Y,last_response
7021226,console,-0.474705,1.094874,0.258717,0.503144,1.507025,-1.408898,0.496110,-0.342881,-0.794755,...,1,0.0,1,0.4154,0.1731,0.4112,0.0003,0.303781,1,0
6836703,consist,-0.470568,0.962991,0.260600,0.480988,1.344495,-1.333904,0.460674,-0.332445,-0.745455,...,1,0.0,1,0.5524,0.1576,0.2895,0.0005,0.563981,1,1
6904211,consistent,-0.404658,0.898675,0.222523,0.423722,1.239092,-1.180395,0.411884,-0.290802,-0.661933,...,1,0.0,1,0.4417,0.1619,0.3961,0.0003,0.457183,1,0
7120550,constant,-0.438270,0.956392,0.241749,0.456376,1.321626,-1.270601,0.442882,-0.313167,-0.712319,...,1,0.0,1,0.4470,0.1674,0.3853,0.0003,0.443431,1,0
7233272,constituent,-0.465048,1.062780,0.253318,0.490704,1.463766,-1.374023,0.482891,-0.335000,-0.774168,...,1,0.0,1,0.3894,0.2085,0.4019,0.0003,0.367841,1,0


In [58]:
dataset.columns

Index(['spelling', 'embed1', 'embed2', 'embed3', 'embed4', 'embed5', 'embed6',
       'embed7', 'embed8', 'embed9', 'embed10', 'embed11', 'embed12',
       'embed13', 'embed14', 'embed15', 'embed16', 'embed17', 'embed18',
       'embed19', 'embed20', 'embed21', 'embed22', 'embed23', 'embed24',
       'embed25', 'embed26', 'embed27', 'embed28', 'embed29', 'embed30',
       'embed31', 'embed32', 'embed33', 'embed34', 'embed35', 'embed36',
       'embed37', 'embed38', 'embed39', 'embed40', 'embed41', 'embed42',
       'embed43', 'embed44', 'embed45', 'embed46', 'embed47', 'embed48',
       'embed49', 'embed50', 'embed51', 'embed52', 'embed53', 'embed54',
       'embed55', 'embed56', 'embed57', 'embed58', 'embed59', 'embed60',
       'embed61', 'embed62', 'embed63', 'embed64', 'uid', 'register_time',
       'wid', 'difficulty', 'review_time', 'response', 'study_method',
       'repeat_time_inday', 'real_interval_history_byday',
       'last_real_interval_byday', 'real_delta_interval_within

### create a new feature for estimate the user tenure

In [59]:
dataset['user_tenure'] = (dataset["review_time"] -dataset["register_time"]).dt.days

### create the dataframe , which is ready for be transformed to a sequential list

In [60]:
df_encoded = dataset[list(KGembedding.columns[1:]) +['Y','spelling','review_time','uid', 'user_tenure','rsp1_rate',
       'rsp2_rate', 'rsp3_rate', 'rsp4_rate', 'acknowledge_rate', 'difficulty',  'study_method', 'repeat_time_inday',
       'real_delta_interval_within_inday', 'review_day_th',
       'delta_t_between_inday', 'delta_t_within_inday', 'repeat_time_history',
       'delta_interval_within_inday']]
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23656968 entries, 7021226 to 29463786
Data columns (total 83 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   embed1                            float32       
 1   embed2                            float32       
 2   embed3                            float32       
 3   embed4                            float32       
 4   embed5                            float32       
 5   embed6                            float32       
 6   embed7                            float32       
 7   embed8                            float32       
 8   embed9                            float32       
 9   embed10                           float32       
 10  embed11                           float32       
 11  embed12                           float32       
 12  embed13                           float32       
 13  embed14                           float32       
 14  embed15   

In [61]:
df_encoded = df_encoded.sort_values(by=['uid', 'review_time'])

In [62]:
#check if any null value in any row
rows_with_null = df_encoded[df_encoded.isnull().any(axis=1)]
rows_with_null

,embed1,embed2,embed3,embed4,embed5,embed6,embed7,embed8,embed9,embed10,...,acknowledge_rate,difficulty,study_method,repeat_time_inday,real_delta_interval_within_inday,review_day_th,delta_t_between_inday,delta_t_within_inday,repeat_time_history,delta_interval_within_inday


In [ ]:
#df_encoded.to_csv("./data/df_encoded_KG_0905.csv",index=None)

In [64]:
del dataset

#### We have over 24 million study records of 3.3W unique users with over 6K words

### transform the dataframe to a sequncial list, each element is a list of a user's sequential study records, each record has X features
#### Consider the list will be very large, I have to optimize it using deque() and seperate chunks

## data read and transforming

In [9]:
df_encoded = pd.read_csv("./data/df_encoded_KG_0905.csv")

In [10]:
valid_uids= list(set(df_encoded.uid))

In [11]:

from collections import deque
chunks =200 # chunk size 
sub_uids=[valid_uids[i:i+chunks] for i in range (0, len(valid_uids),chunks)]
print(len(sub_uids))

169


In [12]:
import time
import itertools
import sys
import pickle
starttime=time.time()

src_data=[]
tgt_data=[]

for i,sub_uid in enumerate(sub_uids):
    print('loop',i)
    try:
        sub_df_encoded = df_encoded[df_encoded.uid.isin(sub_uid)]
        sub_src_data = [group.drop(columns=['uid',"Y",'spelling','review_time']).apply(list, axis=1).tolist() for _, group in tqdm(sub_df_encoded.groupby('uid'), desc='Processing data')]   
        src_data.append(sub_src_data)

        sub_tgt_data = [[elem[0] for elem in group.drop(columns=['uid','spelling','review_time']).apply(list, axis=1).tolist() ]for _, group in tqdm(sub_df_encoded[["uid",'spelling','review_time',"Y"]].groupby('uid'), desc='Processing data')]
        tgt_data.append(sub_tgt_data)


#         with open(f'./data/sub_src_data_list_{i}.pkl', 'wb') as file1:
#             pickle.dump(sub_src_data, file1)

#         with open(f'./data/sub_tgt_data_list_{i}.pkl', 'wb') as file2:
#             pickle.dump(sub_tgt_data, file2)
        #size_in_bytes = sys.getsizeof(src_data)
        #size_in_gigabytes = size_in_bytes / (1024 * 1024 * 1024)
        #print(f"The list takes up {size_in_gigabytes:.2f} GB.")
        print('total time',time.time() -starttime)
        #print(memory_usage())

    except: 
        print('error happens at loop',i )
        sub_df_encoded.to_csv(f"./data/sub_df_encoded_{i}.csv",index=None)
        
src_data =  list(itertools.chain(*src_data))
tgt_data =list(itertools.chain(*tgt_data))

print(memory_usage())

loop 0


Processing data: 100%|██████████| 200/200 [00:00<00:00, 406.84it/s]


total time 2.368722677230835
loop 1


Processing data: 100%|██████████| 200/200 [00:00<00:00, 443.16it/s]


total time 4.445590019226074
loop 2


...

total time 388.6738955974579
loop 167


Processing data: 100%|██████████| 200/200 [00:00<00:00, 456.09it/s]


total time 390.55384278297424
loop 168


Processing data: 100%|██████████| 100/100 [00:00<00:00, 435.03it/s]

total time 391.7118365764618
Total Memory: 1007.45 GB | Used Memory: 917.95 GB | Available Memory: 899.42 GB


In [13]:
print(len(src_data), len(tgt_data))

33700 33700


In [14]:
# Split data into training and testing sets
src_train, src_test, tgt_train, tgt_test = train_test_split(src_data, tgt_data, test_size=0.2, random_state=0)

# Create datasets
train_dataset = MyDataset(src_train, tgt_train)
test_dataset = MyDataset(src_test, tgt_test)



In [15]:
# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)


### model with KG training 

In [16]:
# hyperparamter tuning for our model
# param_grid = {
#     'embedding_size': [32, 64, 128],
#     'num_heads': [2, 4, 8],
#     'num_layers': [ 2, 3, 4],
#     'dropout': [0.1, 0.2, 0.3],
#     'lr': [1e-3, 1e-4, 1e-5]
# }

# from sklearn.metrics import f1_score
# import itertools

# # Helper function to get all parameter combinations
# def get_param_combinations(param_grid):
#     keys = param_grid.keys()
#     values = (param_grid[key] for key in keys)
#     for instance in itertools.product(*values):
#         yield dict(zip(keys, instance))


In [17]:
import torch.nn as nn
# Transformer model
KG_dims =64
KG_compress_dims = 16
input_dim = len(src_train[0][0]) -KG_dims + KG_compress_dims
embedding_size = 16*2
num_heads =4
num_layers = 2
dropout = 0.1

model= TransformerKGModel(KG_dims, KG_compress_dims, input_dim, embedding_size, num_heads, num_layers, dropout)

# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs!")
#     model = nn.DataParallel(model)

model.to(device)


criterion = nn.BCELoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
#next(model.parameters()).device


# Training Loop
num_epochs = 1000
patience = 30  # number of epochs to wait for improvement before stopping
min_delta = 0.001  # minimum improvement required to reset patience counter

#best_loss = None
#patience_counter = 0


best_f1 = None
patience_counter = 0
best_AUC=None

best_model_AUC = 0
best_model = None



In [18]:
# def check_memory(src):
#     bytes_per_element = src.element_size() # Returns the size of a single element in bytes.
#     num_elements = src.numel() # Returns the total number of elements in the tensor.
#     total_memory_bytes = bytes_per_element * num_elements
        
#     total_memory_kB = total_memory_bytes / 1024
#     total_memory_MB = total_memory_kB / 1024
#     total_memory_GB = total_memory_MB / 1024
#     print(total_memory_GB) 


In [19]:

start_time = time.time()
for epoch in range(num_epochs):
    #print("epoch:",epoch)
    epoch_loss = 0
    model.train()

    
    # # # # # # ## # #
    # traning 
    # # # # # # ## # #
    for i,(src, tgt) in enumerate(train_dataloader):
        #print("dataloader:" ,i )
        src = src.to(device)
        tgt = tgt.to(device)
    
        #print("src")
        #check_memory(src)
        # Create masks
        #print("Create masks:" ,i )
        src_key_padding_mask, future_mask = create_masks(src)
        
        #print("src_key_padding_mask")
        #check_memory(src_key_padding_mask)
        #print("future_mask")
       # check_memory(future_mask) 
        
        src_key_padding_mask = src_key_padding_mask.to(device)
        
        future_mask = future_mask.to(device)

        #print("Create masks done:" ,i )
        # Pass your sequences and mask through the model
        output = model(src, src_key_padding_mask, future_mask)
        # Calculate loss, perform backpropagation and update weights
        tgt_mask = (tgt != -1).to(device)
        loss = criterion(output[tgt_mask], tgt[tgt_mask])
        loss = loss.mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    epoch_loss /= len(train_dataloader)  # calculate mean epoch loss
    
    # Early stopping check
    #if best_loss is None or epoch_loss < best_loss - min_delta:
    #    best_loss = epoch_loss
    #    patience_counter = 0
    #else:
    #    patience_counter += 1
    #    if patience_counter >= patience:
    #        print(f'Early stopping on epoch {epoch}')
    #        break
    
    # # # # # # ## # #
    # Testing Loop
    # # # # # # ## # #
    model.eval()
    with torch.no_grad():
        all_probs, all_preds, all_targets = [], [],[]
        for src, tgt in test_dataloader:
            src = src.to(device)
            
            tgt = tgt.to(device)
            # Create masks
            src_key_padding_mask = (src == 0).all(axis=-1).to(device)

            # Pass your sequences and mask through the model
            output = model(src, src_key_padding_mask)

            # Save predictions and targets
            tgt_mask = (tgt != -1).to(device)
            probs = output[tgt_mask].cpu().numpy()
            preds = (output > 0.5).float()
            all_probs.extend(probs)
            all_preds.extend(preds[tgt_mask].cpu().numpy())
            all_targets.extend(tgt[tgt_mask].cpu().numpy())

        # Compute metrics
        accuracy = accuracy_score(all_targets, all_preds)
        f1 = f1_score(all_targets, all_preds)
        roc_auc = roc_auc_score(all_targets, all_probs) if len(np.unique(all_targets)) > 1 else np.nan
        avg_precision = average_precision_score(all_targets, all_probs)

    epoch_time= time.time() - start_time
    print(f"Epoch: {epoch}, timecost:{epoch_time:.2f} seconds, Loss: {epoch_loss}, Accuracy: {accuracy}, F1 Score: {f1}, ROC AUC: {roc_auc}, Average Precision Score: {avg_precision}")

    # # # # # # # ## # #
        # Early stopping check , find the best AUC 
     # # # # # # # # # # # # #
    
            
    if best_AUC is None or roc_auc > best_AUC + min_delta:
        best_AUC = roc_auc
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f'Early stopping on epoch {epoch}')
            break



    if best_AUC > best_model_AUC:
        best_model_AUC = best_AUC
        best_model = model
        torch.save(best_model.state_dict(), "./data/memory_transformer_KG_v2_0905.pth")
    print("best AUC:",best_model_AUC)

    
    
#     if best_f1 is None or f1 > best_f1 + min_delta:
#         best_f1 = f1
#         patience_counter = 0
#     else:
#         patience_counter += 1
#         if patience_counter >= patience:
#             print(f'Early stopping on epoch {epoch}')
#             break

            



Epoch: 0, timecost:201.07 seconds, Loss: 0.5571009640347002, Accuracy: 0.7339584279275215, F1 Score: 0.8293229022363893, ROC AUC: 0.7206060565306357, Average Precision Score: 0.8560682660220055
best AUC: 0.7206060565306357
Epoch: 1, timecost:396.31 seconds, Loss: 0.5374710150040929, Accuracy: 0.7329229186866422, F1 Score: 0.8203643335820736, ROC AUC: 0.7341519258328222, Average Precision Score: 0.8628554223176953
best AUC: 0.7341519258328222
Epoch: 2, timecost:588.64 seconds, Loss: 0.5317547624295119, Accuracy: 0.7390309653392809, F1 Score: 0.8232074265047301, ROC AUC: 0.7480338738271014, Average Precision Score: 0.8715178712349129
best AUC: 0.7480338738271014
Epoch: 3, timecost:780.47 seconds, Loss: 0.5252390283682226, Accuracy: 0.7393770506558612, F1 Score: 0.8247357848393027, ROC AUC: 0.743585941421181, Average Precision Score: 0.8680181645391933
best AUC: 0.7480338738271014
Epoch: 4, timecost:971.82 seconds, Loss: 0.5236478974981902, Accuracy: 0.7362893273246612, F1 Score: 0.819375

### Save model and data

In [20]:
import torch

# Assume you have a model instance called 'model'
torch.save(best_model.state_dict(), "./data/memory_transformer_KG_v2_0905.pth")


In [47]:
# with open(f'./data/src_data_KG.pkl', 'wb') as file1:
#     pickle.dump(src_data, file1)
    
# with open(f'./data/tgt_data_KG.pkl', 'wb') as file1:
#     pickle.dump(tgt_data, file1)

## release memory in GPU

In [20]:
import torch
import gc

# Function to check if an object is a tensor on the GPU
def is_tensor_on_gpu(obj):
    return torch.is_tensor(obj) and obj.is_cuda

# Inspect all objects and delete tensors on GPU
for obj in gc.get_objects():
    try:
        if is_tensor_on_gpu(obj):
            del obj
    except Exception as e:
        # Handle any exception that occurs during deletion
        print(f"Error while deleting object: {e}")

# Clear up all unused memory
torch.cuda.empty_cache()

# Optional: run garbage collector
gc.collect()


/home/ycsong/zhai0045/.conda/envs/cz/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py:293: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


1401

In [48]:
# del src_data
# del tgt_data